## Scripts

`clean_data.py`: This Python file contains all the functions used to clean the geographic fields, the race and ethnicity columns, and action taken columns, among others. It also finds and flags co-applicants among five different fields.

`categorize_data.py`: This Python file contains all the functions that standardize the columns that are used in the regression, including debt-to-income ratio, combined loan-to-value ratio, among others.

`use_regression.py`: This Python file contains all the functions needed to run the regression and other statistical tests.

[Download from GitHub](https://github.com/the-markup/investigation-redlining/tree/main/utils). Code to download in your notebook is included below.

In [1]:
# code to download the file within your Python IDE
import json, requests, urllib, urllib.request
urllib.request.urlretrieve("https://raw.githubusercontent.com/the-markup/investigation-redlining/main/utils/categorize_data.py", "categorize_data.py")
urllib.request.urlretrieve("https://raw.githubusercontent.com/the-markup/investigation-redlining/main/utils/clean_data.py", "clean_data.py")
urllib.request.urlretrieve("https://raw.githubusercontent.com/the-markup/investigation-redlining/main/utils/use_regression.py", "use_regression.py")

('use_regression.py', <http.client.HTTPMessage at 0x7f0ecc2560b0>)

In [2]:
from categorize_data import *
from clean_data import *
from use_regression import *

# Data

## Census Data

`counties`
- We used 2019 American Community Survey data for the property values for each county in the country––table B25077. We downloaded the data from the Census and included the raw dataset.

`metro`
- We used 2019 American Community Survey data for the metro area populations, which we downloaded from the Census website and acquired through the Census API.

`demo`
- We used a Census dataset that lists all counties in the country and the respective metro area that they belong to. That raw dataset is included here. We used this dataset to map counties in HMDA data to their respective metro areas while incorporating the population categories for each metro area.


In [3]:
import pandas as pd
counties = pd.read_csv("https://raw.githubusercontent.com/the-markup/investigation-redlining/main/data/census_data/county_to_metro_crosswalk/clean/all_counties_210804.csv")
metro = pd.read_csv("https://raw.githubusercontent.com/the-markup/investigation-redlining/main/data/census_data/metro_area_pop/raw/metro_division_pop2019.csv")
propValue = pd.read_csv("https://raw.githubusercontent.com/the-markup/investigation-redlining/main/data/census_data/property_values/ACSDT5Y2019.B25077_data_with_overlays_2021-06-23T115616.csv")
demo = pd.read_csv("https://raw.githubusercontent.com/the-markup/investigation-redlining/main/data/census_data/racial_ethnic_demographics/clean/tract_race_pct2019_210204.csv")

## CFPB Data

Original 2019 HDMA source data from the [CFPB website](https://ffiec.cfpb.gov/data-publication/dynamic-national-loan-level-dataset/2019).
- [Data dictionary](https://ffiec.cfpb.gov/documentation/publications/loan-level-datasets/public-lar-schema)
- [Other field-level documentation](https://ffiec.cfpb.gov/documentation/publications/loan-level-datasets/lar-data-fields)

The raw data here is over `6 GB`. We'll work with the output of the reporting team's filtering/reshaping. The broad strokes of their workflow....
- Standardize data
- Standardize applicant and co-applicant race/ethnicity
- Standardize credit models
- Standardize co-applicant info
- Standardize outcomes
- Connect lender info to mortgage info

To unpack the full data processing workflow:
- [Reporting team's Jupyter Notebook](https://github.com/the-markup/investigation-redlining/blob/main/notebooks/process/1_clean_data.ipynb)
- [Prof. Walden's version of their notebook](https://colab.research.google.com/drive/1406la8gg4v7u8LBstU9Ec1GQrKVNe1Da?usp=sharing)

In [ ]:
import pandas as pd # import
hmda19_df2 = pd.read_csv("output.csv", dtype=str) # load data
hmda19_df2 # inspect data

In [ ]:
hmda19_df2.info() # inspect data

## Lender Data

The reporting team also used an `lei` lookup table with additional info on the lenders.
- [Link to this data](https://ffiec.cfpb.gov/data-publication/snapshot-national-loan-level-dataset/2019)

In [ ]:
lenders = pd.read_csv("https://raw.githubusercontent.com/the-markup/investigation-redlining/main/data/supplemental_hmda_data/cleaned/lender_definitions_em210513.csv")
lenders # inspect output

# Data Cleaning

## 1- Merge Lender Info

In [ ]:
lender_def2 = lenders[['lei', 'lar_count', 'assets', 'lender_def', 'con_apps']].copy()
lender_def2.head(1)

In [ ]:
hmda19_df2Merged = pd.merge(hmda19_df2, lender_def2, how = 'left', on='lei')
hmda19_df2Merged

Every record in HMDA data has a lender match. There are no missing values after the join.

In [ ]:
hmda19_df2Merged['lar_count'].isnull().values.sum()

Only 30,000 records, less than one percent,  in overall HMDA data come from no definitions for lenders.
- 1: Banks
- 2: Credit Union
- 3: Independent Mortgage Companies
- 4: No definition

In [ ]:
print(hmda19_df2Merged['lender_def'].value_counts(dropna = False, normalize = True) * 100)

## 2- Add Metro Definitions

In [ ]:
counties.info() # inspect counties df

In [ ]:
counties2 = counties[['fips_state_code', 'fips_county_code', 'metro_code', 'metro_type_def',
                            'metro_percentile']].copy()

counties2 = counties2.rename(columns = {'fips_state_code': 'state_fips',
                                              'fips_county_code': 'county_fips'})

counties2.head(1)

Majority of applications come from metros in the 80th percentile or larger ones.

- 111: Micro
- 000: No Metro
- 99: 99th percentile
- 9: 90th percentile

In [ ]:
hmda19_df2Merged.info()

In [ ]:
counties2 = counties2.astype({'state_fips':str, 'county_fips':str}) # convert data type for merge
counties2.info()

In [ ]:
hmda19_df2Merged2 = pd.merge(hmda19_df2Merged, counties2, how = 'left', on = ['state_fips', 'county_fips'])

hmda19_df2Merged2['metro_percentile'].value_counts(dropna = False, normalize = True) * 100

## 3-Add Property Value By County

In [ ]:
propValue.info() # inspect property value info

In [ ]:
prop_values_df2 = propValue[(propValue['GEO_ID'] != 'id')]

prop_values_df3 = prop_values_df2.rename(columns = {'B25077_001E': 'median_value',
                                                    'B25077_001M': 'median_value_moe'})

prop_values_df3['state_fips'] = prop_values_df3['GEO_ID'].str[9:11]
prop_values_df3['county_fips'] = prop_values_df3['GEO_ID'].str[11:]

prop_values_df4 = prop_values_df3[['state_fips', 'county_fips', 'median_value']].copy()


prop_values_df4.info()

Convert property value to numeric
- No property value for these two counties

In [ ]:
prop_values_df4[(prop_values_df4['median_value'] == '-')]

In [ ]:
prop_values_df4.loc[(prop_values_df4['median_value'] != '-'), 'median_prop_value'] = prop_values_df4['median_value']
prop_values_df4.loc[(prop_values_df4['median_value'] == '-'), 'median_prop_value'] = np.nan
prop_values_df4['median_prop_value'] = pd.to_numeric(prop_values_df4['median_prop_value'])

prop_values_df4[(prop_values_df4['median_prop_value'].isnull())]

In [ ]:
hmda19_df2 = pd.merge(hmda19_df2, prop_values_df4, how = 'left', on = ['state_fips', 'county_fips'])

In [ ]:
hmda19_df2.loc[(hmda19_df2['property_value'] != 'Exempt'), 'prop_value'] = hmda19_df2['property_value']

hmda19_df2.loc[(hmda19_df2['property_value'] == 'Exempt'), 'prop_value'] = np.nan

hmda19_df2['prop_value'] = pd.to_numeric(hmda19_df2['prop_value'])

## 4-Race & Ethnicity By Census Tract

In [ ]:
demo.info() # inspect df

In [ ]:
demo['white_pct'] = pd.to_numeric(demo['white_pct'])

demo['census_tract'] = demo['state'] + demo['county'] + demo['tract']

demo2 = demo[['census_tract', 'total_estimate', 'white_pct', 'black_pct', 'native_pct', 'latino_pct',
                    'asian_pct', 'pacislander_pct', 'othercb_pct', 'asiancb_pct']].copy()

demo2.sample(2, random_state = 303)

White gradiant

In [ ]:
demo2.loc[(demo2['white_pct'] > 75), 'diverse_def'] = '1'

demo2.loc[(demo2['white_pct'] <= 75) & (demo2['white_pct'] > 50), 'diverse_def'] = '2'

demo2.loc[(demo2['white_pct'] <= 50) & (demo2['white_pct'] > 25), 'diverse_def'] = '3'

demo2.loc[(demo2['white_pct'] <= 25), 'diverse_def'] = '4'

demo2.loc[(demo2['white_pct'].isnull()), 'diverse_def'] = '5'

demo2['diverse_def'].value_counts(dropna = False)

- 0: No census data there
- NaN: Records that don't find a match in the census data

In [ ]:
demo2 = demo2.astype({'census_tract':str})
hmda19_df2 = pd.merge(hmda19_df2, demo2, how = 'left', on = ['census_tract'])

Convert the NaN to 0's

In [ ]:
hmda19_df2.loc[(hmda19_df2['diverse_def'].isnull()), 'diverse_def'] = '0'

hmda19_df2['diverse_def'].value_counts(dropna = False)

## 5-Debt to Income Ratio

In [ ]:
dti_df = pd.DataFrame(hmda19_df2['debt_to_income_ratio'].value_counts(dropna = False)).reset_index().\
         rename(columns = {'index': 'debt_to_income_ratio', 'debt_to_income_ratio': 'count'})

### Convert the nulls for cleaning purposes
dti_df = dti_df.fillna('null')

dti_df.head(2)

In [ ]:
### Running function to organize debt-to-income ratio
dti_df['dti_cat'] = dti_df.apply(setup_dti_cat, axis = 1)

dti_df.head(2)

In [ ]:
### Drop count column and replace the null values back to NaN
dti_df2 = dti_df.drop(columns = ['count'], axis = 1)
dti_df2 = dti_df2.replace('null', np.nan)

dti_df2.head(2)

A third of entire dataset is null, when it comes to DTI ratio.

In [ ]:
hmda19_df2 = pd.merge(hmda19_df2, dti_df2, how = 'left', on = ['debt_to_income_ratio'])

hmda19_df2['dti_cat'].value_counts(dropna = False, normalize = True) * 100

## 6-Loan-to-value ratio

In [ ]:
cltv_df = pd.DataFrame(hmda19_df2['combined_loan_to_value_ratio'].value_counts(dropna = False)).reset_index().\
          rename(columns = {'index': 'combined_loan_to_value_ratio', 'combined_loan_to_value_ratio': 'count'})

### Convert cltv to numeric
cltv_df.loc[(cltv_df['combined_loan_to_value_ratio'] != 'Exempt'), 'cltv_ratio'] =\
            cltv_df['combined_loan_to_value_ratio']

cltv_df['cltv_ratio'] = pd.to_numeric(cltv_df['cltv_ratio'])

Downpayment Flag
- 1: 20 percent or more downpayment
- 2: Less than 20 percent
- 3: Nulls

In [ ]:
cltv_df['downpayment_flag'] = cltv_df.apply(categorize_cltv, axis = 1)
cltv_df2 = cltv_df.drop(columns = ['count', 'cltv_ratio'], axis = 1)


hmda19_df2 = pd.merge(hmda19_df2, cltv_df2, how = 'left', on = ['combined_loan_to_value_ratio'])
hmda19_df2['downpayment_flag'].value_counts(dropna = False)

## 7-Property Value Ratio Z-Score

Property value ratios are more normally distributed than raw property values. Because there's they are normally distributed below the 10th ratio, I will use the z-scores and place them into buckets based on those z-scores.

In [ ]:
property_value_df = pd.DataFrame(hmda19_df2.groupby(by = ['state_fips', 'county_fips', 'property_value',
                    'prop_value', 'median_prop_value'], dropna = False).size()).reset_index().\
                     rename(columns = {0: 'count'})

In [ ]:
property_value_df['property_value_ratio'] = property_value_df['prop_value'].\
                                            div(property_value_df['median_prop_value']).round(3)

property_value_df['prop_zscore'] = property_value_df.apply(calculate_prop_zscore, axis = 1).round(3)

property_value_df['prop_value_cat'] = property_value_df.apply(categorize_property_value_ratio, axis = 1)

property_value_df.sample(3, random_state = 303)

In [ ]:
property_value_df2 = property_value_df[['state_fips', 'county_fips', 'property_value',
                                        'median_prop_value', 'property_value_ratio', 'prop_zscore',
                                        'prop_value_cat']].copy()

In [ ]:
hmda19_df2 = pd.merge(hmda19_df2, property_value_df2, how = 'left', on = ['state_fips', 'county_fips',
                     'property_value', 'median_prop_value'])

## 8-Applicant Age

- [9999](https://s3.amazonaws.com/cfpb-hmda-public/prod/help/2018-public-LAR-code-sheet.pdf): No Co-applicant
- 8888: Not Applicable

In [ ]:
age_df = pd.DataFrame(hmda19_df2['applicant_age'].value_counts(dropna = False)).reset_index().\
         rename(columns = {'index': 'applicant_age', 'applicant_age': 'count'})

In [ ]:
age_df['applicant_age_cat'] = age_df.apply(categorize_age, axis = 1)

age_df = age_df.drop(columns = ['count'], axis = 1)

#### Age Categories
- 1: Less than 25
- 2: 25 through 34
- 3: 35 through 44
- 4: 45 through 54
- 5: 55 through 64
- 6: 65 through 74
- 7: Greater than 74
- 8: Not Applicable

In [ ]:
hmda19_df2 = pd.merge(hmda19_df2, age_df, how = 'left', on = ['applicant_age'])

hmda19_df2['applicant_age_cat'].value_counts(dropna = False)

## 9-Income and Loan Amount Log

In [ ]:
hmda19_df2['income'] = pd.to_numeric(hmda19_df2['income'])
hmda19_df2['loan_amount'] = pd.to_numeric(hmda19_df2['loan_amount'])

hmda19_df2['income_log'] = np.log(hmda19_df2['income'])
hmda19_df2['loan_log'] = np.log(hmda19_df2['loan_amount'])

## 10-Applicant Sex
- 1: Male
- 2: Female
- 3: Information not provided
- 4: Not Applicable
- 5: No Co-Applicable
- 6: Marked Both

In [ ]:
sex_df = pd.DataFrame(hmda19_df2['applicant_sex'].value_counts(dropna = False)).reset_index().\
         rename(columns = {'index': 'applicant_sex', 'applicant_sex': 'count'})

In [ ]:
sex_df = sex_df.drop(columns = ['count'], axis = 1)

sex_df['applicant_sex_cat'] = sex_df.apply(categorize_sex, axis = 1)

#### New applicant sex categories
- 1: Male
- 2: Female
- 3: Not applicable
- 4: Makred both sexes

In [ ]:
hmda19_df2 = pd.merge(hmda19_df2, sex_df, how = 'left', on = ['applicant_sex'])

hmda19_df2['applicant_sex_cat'].value_counts(dropna = False)

## 11-Automated Underwiting systems
- 1: Only one AUS was used
- 2: Same AUS was multiple times
- 3: Different AUS were used
- 4: Exempt

In [ ]:
hmda19_df2['aus_cat'].value_counts(dropna = False)

In [ ]:
underwriter_df = pd.DataFrame(hmda19_df2.groupby(by = ['aus_1', 'aus_cat']).size()).reset_index().\
                 rename(columns = {0: 'count'})

underwriter_df['main_aus'] = underwriter_df.apply(categorize_underwriter, axis = 1)

underwriter_df = underwriter_df.drop(columns = ['count'], axis = 1)

#### Main Aus
- 1: Desktop Underwriter
- 2: Loan Prospector
- 3: Technology Open to Approved Lenders
- 4: Guaranteed Underwriting System
- 5: Other
- 6: No main Aus
- 7: Not Applicable

In [ ]:
hmda19_df2 = pd.merge(hmda19_df2, underwriter_df, how = 'left', on = ['aus_1', 'aus_cat'])

hmda19_df2['main_aus'].value_counts(dropna = False)

## 12- Loan Term

In [ ]:
loanterm_df = pd.DataFrame(hmda19_df2['loan_term'].value_counts(dropna = False)).reset_index().\
              rename(columns = {'index': 'loan_term', 'loan_term': 'count'})

loanterm_df.loc[(loanterm_df['loan_term'] != 'Exempt'), 'em_loan_term'] = loanterm_df['loan_term']

loanterm_df['em_loan_term'] = pd.to_numeric(loanterm_df['em_loan_term'])

In [ ]:
loanterm_df['mortgage_term'] = loanterm_df.apply(categorize_loan_term, axis = 1)

loanterm_df = loanterm_df.drop(columns = ['count', 'em_loan_term'])

#### Mortgage Term
- 1: 30 year mortgage
- 2: Less than 30 years
- 3: More than 30 years
- 4: Not applicable

In [ ]:
hmda19_df2 = pd.merge(hmda19_df2, loanterm_df, how = 'left', on = ['loan_term'])

hmda19_df2['mortgage_term'].value_counts(dropna = False)

## 13-Tract MSA Income Percentage

In [ ]:
tractmsa_income_df = pd.DataFrame(hmda19_df2['tract_to_msa_income_percentage'].value_counts(dropna = False)).\
                     reset_index().rename(columns = {'index': 'tract_to_msa_income_percentage',
                                                     'tract_to_msa_income_percentage': 'count'})

tractmsa_income_df['tract_msa_ratio'] = pd.to_numeric(tractmsa_income_df['tract_to_msa_income_percentage'])

In [ ]:
tractmsa_income_df['lmi_def'] = tractmsa_income_df.apply(categorize_lmi, axis = 1)

tractmsa_income_df = tractmsa_income_df.drop(columns = ['count', 'tract_msa_ratio'], axis = 1)

LMI Definition
- 1: Low
- 2: Moderate
- 3: Middle
- 4: Upper
- 5: None

In [ ]:
hmda19_df2 = pd.merge(hmda19_df2, tractmsa_income_df, how = 'left', on = ['tract_to_msa_income_percentage'])

hmda19_df2['lmi_def'].value_counts(dropna = False)

## 14-Filter:

#### For Conventional and FHA loans that first-lien, one-to-four unit, site built unites for home purchase where the applicant is going to live in that property

In [ ]:
one_to_four = ['1', '2', '3', '4']

hmda19_df2 = hmda19_df2[((hmda19_df2['loan_type'] == '1') | (hmda19_df2['loan_type'] == '2'))\
                      & (hmda19_df2['occupancy_type'] == '1') &\
                        (hmda19_df2['total_units'].isin(one_to_four)) &\
                        (hmda19_df2['loan_purpose'] == '1') &\
                        (hmda19_df2['action_taken'] != '6') &\
                        (hmda19_df2['construction_method'] == '1') &\
                        (hmda19_df2['lien_status'] == '1') &\
                        (hmda19_df2['business_or_commercial_purpose'] != '1')].copy()

print('hmda19_df2: ' + str(len(hmda19_df2)))
print('hmda19_df2: ' + str(len(hmda19_df2)))

## 15-Output

In [ ]:
hmda19_df2.to_csv("output2.csv", index=False)
print(hmda19_df2)